In [1]:
import re
import mmap
from pprint import pprint
import json


In [13]:
class Keyword():
    def __init__(self, kw): self.__dict__.update(kw)
    def get_name(self):     return self.ID if 'ID' in self.__dict__ else self.IC
    def get_category(self): return self.CA if 'ID' in self.__dict__ else self.IC
    def is_category(self):  return 'IC' in self.__dict__
    def is_keyword(self):   return 'ID' in self.__dict__
    
def parse_keywlist(path):

    id_re = re.compile(rb'(ID|IC) \s+ ([^.]+) .\s+  AC \s+ (KW-\d{4}) \s+', re.X)
    multiline_re = re.compile(rb'//\s+|(\w\w)\s+([^\.]+)\.\s+', re.S)
    kw_split_re = re.compile(rb'\s*[;:]\s*')

    kw_name = {}
    kw_ac = {}

    with open(path, 'rb+') as file:
        mem = mmap.mmap(file.fileno(), 0)
        pos = re.search(b'^ID', mem, re.M).start()

        while True:
            m = id_re.match(mem, pos)
            if not m:
                break

            id_type, id_value, AC = m.groups()
            rec = Keyword( {id_type.decode():id_value, 'AC': AC, 'GO':[], 'HI':set(), 'WW':[]} )


            for m in multiline_re.finditer(mem, m.end()):
                k, v= m.groups()

                if not k: 
                    pos = m.end() # got '//' update pos for the next kw
                    break

                k = k.decode()

                if k=='DE':
                    rec.DE = v.replace(b'\nDE   ', b' ')
                elif k=='SY':
                    rec.SY = [w.strip() for w in v.replace(b'\nSY   ', b'').split(b';') ]
                elif(k=='GO'):
                    rec.GO.append( v[:10] )
                elif(k=='HI'):
                    rec.HI.add( kw_split_re.split(v)[-2] ) # -1 is ID, -2 is predecessor
                elif(k=='CA'):
                    rec.CA = v
                elif(k=='WW'):
                    rec.WW.append(v)

            kw_ac[rec.AC] = rec
            kw_name[ rec.get_name() ] = rec
            
        #pprint(rec)
        #break # debug
    return kw_name, kw_ac

In [17]:
#kw_name, kw_ac = parse_keywlist('data/IN/keywlist.txt')